In [37]:
#数据读取
import xml.dom.minidom
from functools import reduce
import operator
import math
import numpy as np
import pandas as pd
dom = xml.dom.minidom.parse('/Users/zgqq/Documents/研究生/科研/无人叉车/项目资料/VnSimulator.v0.6.18.16/config/HG12/Path.config')

In [76]:
root = dom.documentElement
itemlist = root.getElementsByTagName('Path')
item = itemlist[0]
un=item.getAttribute("No")
Path = {}
for item in itemlist:
    un=int(item.getAttribute("No"))
    ut=item.getAttribute("Detail")
    a = [x.split(',') for x in ut.split(';')]
    b = reduce(operator.add, a)
    Path[un]=b

In [87]:
Path[61][0]

'F'

In [79]:
class arc:
    """
    用来记录arc
    """
    def __init__(self,index,start=[],end=[]):
        self.index = index
        self.start = start
        self.end = end
    def __repr__(self):
        return "["+ str(self.index) +"]"

In [81]:
def contain_arc(arcs,courrent_arc):
    for arc in arcs:
        if arc.start == courrent_arc.start and arc.end == courrent_arc.end:
            return False
        if arc.start == courrent_arc.end and arc.end == courrent_arc.start:
            return False
        
    return True

In [82]:
arc_sum = []
for key in Path:
    k = arc(key,[Path[key][1],Path[key][2]],[Path[key][4],Path[key][5]])
    if(contain_arc(arc_sum,k)):
        arc_sum.append(k)

In [83]:
 = []
for item in arc_sum:
    index_list.append(item.index)
MA=pd.DataFrame(columns=index_list,index=index_list).fillna(0)

In [84]:
darc = np.zeros([len(arc_sum)+1,len(arc_sum)+1])
for i in range(len(arc_sum)):
    for j in range(i,len(arc_sum)):
        if arc_sum[i].end == arc_sum[j].start or arc_sum[i].start == arc_sum[j].end:
            MA[arc_sum[i].index][arc_sum[j].index] = 1
            MA[arc_sum[j].index][arc_sum[i].index] = 1

In [114]:
class Grid:
    def __init__(self, index):
        self.index = index
        self.x = (float(Path[index][1]) + float(Path[index][4]))/2
        self.y = (float(Path[index][2]) + float(Path[index][5]))/2
        self.length = abs(float(Path[index][4]) - float(Path[index][1])) + abs(float(Path[index][5]) - float(Path[index][2]))
        self.time = self.length / car_speed
        self.f = 0
        self.g = 0
        self.h = 0
        self.parent = None

    def init_grid(self, parent, end):
        self.parent = parent
        if parent is not None:
            self.g = parent.g + 1
        else:
            self.g = 1
        self.h = abs(self.x - end.x) + abs(self.y - end.y)
        self.f = self.g + self.h
    def __repr__(self):
        return "["+ str(self.index) +"]"

In [62]:
def contain_grid(grids, index):
    for grid in grids:
        if (grid.index == index):
            return True
    return False

In [63]:
def is_valid_grid(index, open_list=[], close_list=[]):
    """
    判断是否越界和障碍物方法
    :param x:
    :param y:
    :param open_list:
    :param close_list:
    :return:
    """
    # 是否已经在open_list中
    if contain_grid(open_list, index):
        return False
    # 是否已经在close_list中
    if contain_grid(close_list, index):
        return False
    return True

In [94]:
def find_neighbors(grid, open_list=[], close_list=[]):
    """
    寻找当前格子领近格子的方法
    :param grid:
    :param open_list:
    :param close_list:
    :return:
    """
    grid_list = []
    temp_list = []
    for index in index_list:
        if MA[grid.index][index] == 1:
            temp_list.append(index)
        if MA[index][grid.index] == 1:
            temp_list.append(index)  
            
    for index in temp_list:
        if flag[grid.index][index] == 0:
            if is_valid_grid(index, open_list, close_list):
                grid_list.append(Grid(index))
    return grid_list

In [65]:
def find_min_gird(open_list=[]):
    """
    寻找F值最小的格子的方法
    :param open_list:
    :return:
    """
    temp_grid = open_list[0]
    for grid in open_list:
        if grid.f < temp_grid.f:
            temp_grid = grid
    return temp_grid

In [66]:
def a_star_search(start, end):
    """
    寻路方法，找到终点返回终点grid,否则返回None
    :param start:
    :param end:
    :return:
    """

    # 待访问的格子
    open_list = []
    # 已访问的格子
    close_list = []
    # 把起点加入open_list中
    open_list.append(start)
    # 主循环，每一轮检查一个当前方格节点
    while len(open_list) > 0:
        # 在open_list中查找F值最小的节点作为当前方格节点
        current_grid = find_min_gird(open_list)
        # 将F值最小的节点从open_list中删除
        open_list.remove(current_grid)
        # 将F值最小的节点加入到close_list中
        close_list.append(current_grid)
        # 找到当前节点的所有领近节点
        neighbors = find_neighbors(current_grid, open_list, close_list)
        for grid in neighbors:
            if grid not in open_list:
                # 如果当前节点不在open_list中，标记为父节点，并放入open_list中
                grid.init_grid(current_grid, end)
                open_list.append(grid)
            # 如果终点在open_list中，直接返回终点格子
            for grid in open_list:
                if grid.index == end.index:
                    return grid
        # 遍历完open_list,仍然找不到终点，说明还没到终点，返回空
    return None

In [96]:
flag=pd.DataFrame(columns=index_list,index=index_list).fillna(0)

In [133]:
# 设置起始点和终点
start_grid = Grid(105)
end_grid = Grid(93)
# 搜索迷宫终点
result_grid = a_star_search(start_grid, end_grid)
# 回溯迷宫路径
path = []
while result_grid is not None:
    path.append(Grid(result_grid.index))
    result_grid = result_grid.parent
path = list(reversed(path))
print(path)

[[105], [7932], [7931], [7930], [7929], [7928], [93]]


In [101]:
dom3 = xml.dom.minidom.parse('/Users/zgqq/Documents/研究生/科研/无人叉车/项目资料/VnSimulator.v0.6.18.16/config/HG12/tasks-list.config')

In [102]:
root4 = dom3.documentElement

In [103]:
itemlist4 = root4.getElementsByTagName('Group')

In [104]:
task = itemlist4[0].getElementsByTagName('Task')

In [122]:
Task = {}
tlist = []
num = 1
for item in itemlist4:
    tlist = []
    task = item.getElementsByTagName('Task')
    TimeMode = item.getAttribute("TimeMode")
    Time = item.getAttribute("Time")
    TId = item.getAttribute("Id")
    TType = item.getAttribute("Type")
    TPriority = item.getAttribute("Priority")
    AgvId = item.getAttribute("AgvId")
    tlist.append(TId)
    tlist.append(Time)
    ttlist = tlist.copy()
    for i in range(0,2):
        Id = task[i].getAttribute("Id")
        Type = task[i].getAttribute("Type")
        Goal = task[i].getAttribute("Goal")
        ttlist.append(Id)
        ttlist.append(Type)
        ttlist.append(Goal)
    Task[num] = ttlist
    num = num + 1
    ttlist = []

In [123]:
Task[1]

['1', '0.500', '1', 'GET', '1057', '2', 'PUT', '1307']

None


In [128]:
final_path = []
total_time = 0
car_speed = 1.5
start_grid = Grid(105)
for i in range(1,5):
    end = int(Task[i][4])
    end_grid = Grid(end)
    result_grid = a_star_search(start_grid, end_grid)
    path = []
    while result_grid is not None:
        path.append(Grid(result_grid.index))
        result_grid = result_grid.parent
    path = list(reversed(path))
    if final_path == []:
        for i in range(len(path)):
            final_path.append(path[i])
            total_time += path[i].time
    else:
        for i in range(1,len(path)):
            final_path.append(path[i])
            total_time += path[i].time
    final_path.append(['break_time',20+total_time])
    start = int(Task[i][4])
    end = int(Task[i][7])
    start_grid = Grid(start)
    end_grid = Grid(end)
    result_grid = a_star_search(start_grid, end_grid)
    path = []
    while result_grid is not None:
        path.append(Grid(result_grid.index))
        result_grid = result_grid.parent
    path = list(reversed(path))
    if final_path == []:
        for i in range(len(path)):
            final_path.append(path[i])
            total_time += path[i].time
    else:
        for i in range(1,len(path)):
            final_path.append(path[i])
            total_time += path[i].time
    final_path.append(['break_time',20+total_time])
    print(final_path)

[['break_time', 20], ['break_time', 20]]
[['break_time', 20], ['break_time', 20], ['break_time', 20], ['break_time', 20]]


KeyError: 1124